# Purpose
This notebook describes the typical activities carried out at the beginning to a project / thread when customer shares new data. We will be trying to understand the tables, columns and information flow. Typically we also look for data issues and confirm with respective owners for resolution. At the end of this activity, the data sources and their treatment is finalized. Code in this notebook will not be part of the production code.

This data can be downloaded from [here](https://drive.google.com/file/d/14FaUvJuhQLZevvqQcnOZZxvkvKbdPhZ7/view?usp=sharing)

# Initialization

In [24]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import os
import os.path as op
import pandas as pd 
import great_expectations as ge
from dateutil.relativedelta import relativedelta
os.environ['TA_DEBUG'] = "False"
os.environ['TA_ALLOW_EXCEPTIONS'] = "True"

In [26]:
import warnings

warnings.filterwarnings('ignore', message="The sklearn.metrics.classification module", category=FutureWarning)
warnings.filterwarnings('ignore', message=".*title_format is deprecated. Please use title instead.*")

In [27]:
%%time
from ta_lib.core.api import (
    create_context,
    get_package_path,
    display_as_tabs,
    initialize_environment,
    string_cleaning,
    setanalyse,
    merge_expectations
)
import ta_lib.core.api as dataset
import ta_lib.eda.api as analysis
import ta_lib.reports.api as health

Wall time: 0 ns


In [28]:
# Initialization
initialize_environment(debug=False, hide_warnings=True)

OSError: [Errno 22] Invalid argument

# Data

## Background

Customer is a distributor of consumption goods. There are a variety of data files which we would be using. For more refer to the readme in the data archive. Business wants to know whether a particular customer will churn or not with Machine learning. 

Below is an overview of each of the keys in the dataset:

| | Key | Dataset | Details |
|--|--|--|--|
| 1 | opportunity_id | opportunity.xlsb | The monthly primary sales from the manufacturer to each distributor. The column pri_sales_amount indicates the sales. We will use this column for defining churn of a distributor, i.e. whenever the sales turn 0 or negative, the distributor would be defined as churned. The cutoff or reference date we'll use to predict future churn is 2013-05-01 |
| 2 | product_id | product.xlsb | The monthly secondary business, with column sec_netvalues identifying the total sales from each distributor |

In [133]:
%%time
!pip install pyxlsb
opp_xlsb = pd.read_excel('../../data/raw/opportunity/opportunity.xlsb', engine='pyxlsb')
pro_xlsb = pd.read_excel('../../data/raw/opportunity/product.xlsb', engine='pyxlsb')
opp_xlsb.shape, pro_xlsb.shape

Wall time: 55.3 s


((313571, 16), (142431, 8))

In [161]:
print('\nOpportunity Data Null Values:\n')
print(opp_xlsb.isna().sum())
print('\nProduct Data Null Values:\n')
print(pro_xlsb.isna().sum())


Opportunity Data Null Values:

Opportunity ID                     0
Transition To Stage                0
Transition To Timestamp            0
Transition From Stage Name      5250
Transition From Timestamp       5250
Customer Name                      0
Risk Status                   200380
Creation Date                      0
Decision Date                      0
Opportunity Status               826
Opportunity Type               21384
Core Consumption Market            0
Core Product Segment               0
Core Sales Segment                 0
Geography                          0
Core Product Application           0
dtype: int64

Product Data Null Values:

Opportunity ID            0
Product ID                0
Product Segment  Name     0
Product Status           48
Product $                 0
Product Quantity          0
Decision Date             0
Snapshot Time             0
dtype: int64


In [31]:
# opp_xlsb['Creation Date'] = pd.to_datetime(opp_xlsb['Creation Date'], unit='D', origin='1899-12-30')
# opp_xlsb['Decision Date'] = pd.to_datetime(opp_xlsb['Decision Date'], unit='D', origin='1899-12-30')
# pro_xlsb['Decision Date'] = pd.to_datetime(pro_xlsb['Decision Date'], unit='D', origin='1899-12-30')

In [32]:
# opp_xlsb['Transition From Timestamp'] = opp_xlsb['Transition From Timestamp'].fillna(0).astype('int64').copy()
# opp_xlsb['Transition From Stage Name'] = opp_xlsb['Transition From Stage Name'].fillna('Stage 0').astype('object').copy()

In [135]:
opp_xlsb.to_csv('../../data/raw/opportunity/csv/opportunity.csv', index=False)
pro_xlsb.to_csv('../../data/raw/opportunity/csv/product.csv', index=False)

In [136]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)
dataset.list_datasets(context)

['/raw/product',
 '/raw/opportunity',
 '/cleaned/product',
 '/cleaned/opportunity',
 '/processed/merged',
 '/train/merged/features',
 '/train/merged/target',
 '/test/merged/features',
 '/test/merged/target',
 '/score/merged/output']

In [137]:
# Loading all datasets in a loop
data = dict()
for i in dataset.list_datasets(context):
    if '/raw/' in i:
        key_ = i.replace('/raw/','')+'_df'
        data[key_] = dataset.load_dataset(context,i)
        # Standardize column names
        data[key_].columns = string_cleaning(data[key_].columns,lower=True)

## Exploratory Data Analysis

### Shape of Data

In [138]:
(
    pd.DataFrame({x:data[x].shape for x in data.keys()})
    .T
    .rename(columns={0:'rows',1:'columns'})
    .sort_values('rows',ascending=False)
)

,rows,columns
opportunity_df,313571,16
product_df,142431,8


In [173]:
pd.to_datetime(data['opportunity_df']['transition to timestamp'], format='%Y%m%d%H%M%S')

0        2015-02-11 13:00:01
1        2015-02-13 05:00:02
2        2015-02-18 21:00:07
3        2015-03-04 21:00:02
4        2015-03-04 21:00:02
                 ...        
313566   2016-10-25 07:51:57
313567   2016-10-23 13:00:03
313568   2016-10-25 07:51:57
313569   2016-10-23 21:00:04
313570   2016-10-25 07:51:57
Name: transition to timestamp, Length: 313571, dtype: datetime64[ns]

## Variable summary

In [139]:
summaries = [analysis.get_variable_summary(data[x]) for x in data.keys()]
display_as_tabs([(x, summaries[idx]) for idx, x in enumerate(data.keys())])

Tabs
    [0] DataFrame(name='product_df', value=                          ...)
    [1] DataFrame(name='opportunity_df', value=                          ...)

## Merging

### Expected data validation rules

1. Creation Date is before Decision Date.
2. Product in an specific Opportunity cannot have both ('Win Approved','Win Submitted') and ('Cancelled','Deleted').
3. Product Quantity is an integer value.
4. area name in returns_df should be in sec_bpm_df or retail_program_df
5. primary keys
6. No time gaps
7. customer_code in all tables should be subset of doj

#### Great Expectations 

We will leverage the [Great Expectations](https://docs.greatexpectations.io/en/latest/) library for exploring the data quality & validating whether it conforms to expected business rules. 

#### Rule 1 verification

In [140]:
verification_dict = {}
# Rule tests
test_success_flag = True
verification_dict['rule_1_test'] = (data['opportunity_df']['decision date'] - data['opportunity_df']['creation date']).apply(lambda x: x < 0).sum()

test_success_flag = verification_dict['rule_1_test'] == 0

# Rule 1
if test_success_flag:
    print(('Rule {} passed').format(1))
else:
    print(('Rule {} failed').format(1))

Rule 1 failed


In [157]:
verification_dict['rule_1_test']

10956

#### Rule 2 verification

In [151]:
# verification_dict['rule_2_test'] = data['product_df'].groupby(['opportunity id','product id'])
# data['product_df']['product status'] = data['product_df']['product status'].str.split(' ').str[0].replace('Cancelled', 'Lost').replace('Deleted', 'Lost').copy()
# prostat = pd.DataFrame(data['product_df'].groupby(['opportunity id','product id'])['product status'].apply(lambda x: list(set(x))))
prostat['product status']

opportunity id  product id
5               404           [Win Approved, Pending]
19              16377         [Win Approved, Pending]
                16378         [Win Approved, Pending]
                16380         [Win Approved, Pending]
                16381         [Win Approved, Pending]
                                       ...           
42777           35993                       [Pending]
42778           38156                       [Pending]
42781           38151                       [Pending]
42789           38158                       [Pending]
56117           25835         [Pending, Win Approved]
Name: product status, Length: 4384, dtype: object

### Fixes for failed rules

#### Rule 5

In [18]:
df_temp = (
    data['no_obj_df']
    .groupby(pk_rules['no_obj_df'])
    .size()
    .reset_index()
    .rename(columns={0:'size'})
    .query('size > 1')
    .sort_values('size')
)

verification_dict["merge_1"] = merge_expectations(data['no_obj_df'], df_temp, pk_rules['no_obj_df'])
print(verification_dict["merge_1"]["actionable_warnings"])
sample_duplicates = (
    data['no_obj_df']
    .merge(df_temp,on=pk_rules['no_obj_df'])
    .sort_values(['size']+pk_rules['no_obj_df'])
    .drop('size',axis=1)
)
display_as_tabs([
    ('freq',df_temp['size'].value_counts()), 
    ('head',df_temp.head()), 
    ('tail',df_temp.tail()),
    ('sample_duplicates_head',sample_duplicates.head()),
    ('sample_duplicates_tail',sample_duplicates.tail())
])

{'data_type_mismatch': False, 'nulls_warning': True, 'data_loss_B-A': False, 'data_loss_A-B': True}


Tabs
    [0] DataFrame(Series, name='freq')
    [1] DataFrame(name='head', value=     claim_year q...)
    [2] DataFrame(name='tail', value=     claim_year q...)
    [3] DataFrame(name='sample_duplicates_head', value=     claim_year q...)
    [4] DataFrame(name='sample_duplicates_tail', value=     claim_year q...)

##### Data duplicates seems to be problem. Trying by dropping duplicates

In [19]:
(
    data['no_obj_df']
    .drop_duplicates()
    .groupby(pk_rules['no_obj_df'])
    .size()
    .reset_index()
    .rename(columns={0:'size'})
    .query('size > 1')
    .sort_values('size')
).shape

(0, 6)

#### Rule 7

Ideally all the customers present in all data sources are should be present in the doj_df(date of joining data). However there seem to be 284 customers who have data in the business data but dont have a date of joining.

This has to be reported to the client.

However based on EDA we can choose to either select the customers present in both doj_df and ec_df or use ec_df and fill in proxy date of joining for the missing ones.

## Health Analysis

Get an overview of the overall health of your dataset. This is usually quick to compute and hopefully highlights some problems to focus on.



### Summary Plot

Provides a high level summary of the health your dataset.

**Watch out for:**

* too few numeric values
* high % of missing values
* high % of duplicate values
* high % of duplicate columns 

In [152]:
summaries_and_plots = [analysis.get_data_health_summary(data[x], return_plot=True) for x in data.keys()]
plots = [x[1] for x in summaries_and_plots]
display_as_tabs([(x, plots[idx]) for idx, x in enumerate(data.keys())])

Tabs
    [0] HoloViews(Layout, name='product_df')
    [1] HoloViews(Layout, name='opportunity_df')

**Dev NOTES**

<details>
1. Datatypes : We have both numeric and other types. The bulk of them seem to be numeric. `Numeric` is defined to be one of [float|int|date] and the rest are categorized as `Others`. A column is assumed to have `date` values if it has the string `date` in the column name.

**[TODO]** We prob. need more types: integral, float, bool, dates/timestamps, strings. We have this functionality in Dataprocessor.

2. The missing value plot seems to indicate missing values are not present but we do have them. 

**[TODO]** The plot can be improved to better display small values

3. We are looking for duplicate observations (rows in the data). The plot shows the % of rows that are an exact replica of another row (using `df.duplicated`)

4. We are looking for duplicate features (columns in the data).

**[TODO]** The tigerml code seems complicated but it looks like we compare each column against all other similar columns (numeric/categoric) after dropping nans, infs


**[TODO]** We need better data inspectors. The current data inspectors show columns from the dataframe used to construct the plot and **not** the original data. This does not make sense for an end-user who didn't expicitly construct the intermediate data used for the plot. It would be more meaningful to have labels that match the legends (e.g unique_columns:100%, duplicate_columns:0). Also, the y-axis label doesen't tell anything. The x-axis prob. needs an axis (0 to 100%).

</details>

### Missing Values summary

This provides an overall view focussing on amount of missing values in the dataset.

**Watch out for:**
* A few columns have significant number of missing values 
* Most columns have significant number of missing values


In [153]:
summaries_and_plots = [analysis.get_missing_values_summary(data[x], return_plot=True) for x in data.keys()]
plots = [x[1] for x in summaries_and_plots]
display_as_tabs([(x, plots[idx]) for idx, x in enumerate(data.keys())])

Tabs
    [0] HoloViews(Bars, name='product_df')
    [1] HoloViews(Bars, name='opportunity_df')

**Dev notes:**

<details>
    
    * By default, the following are considered missing/NA values : `[np.Nan, pd.NaT, 'NA', None]`
    * additional values can be passed to tigerml (add_additional_na_values)
    * these are applied to all columns.
    
    * some of the above information can be learnt from the data discovery step (see discussion below)
    
</details>

### Duplicate Columns

In [154]:
summaries = [analysis.get_duplicate_columns(data[x]) for x in data.keys()]
display_as_tabs([(x, summaries[idx]) for idx, x in enumerate(data.keys())])

Tabs
    [0] Markdown(str, name='product_df')
    [1] Markdown(str, name='opportunity_df')

### Outlier Checks

In [155]:
summaries = [analysis.get_outliers(data[x]) for x in data.keys()]
display_as_tabs([(x, summaries[idx]) for idx, x in enumerate(data.keys())])

Tabs
    [0] DataFrame(name='product_df', value=                  ('Data S...)
    [1] DataFrame(name='opportunity_df', value=                          ...)

## Health Analysis report

Generate a report that has all the above data in a single html. This could be useful to submit to a client

In [156]:
os.environ['DEBUG'] = 'true'
health_reports_gen = [health.summary_report(data[x],save_path=r'./health_report_'+x+'.html') for x in data.keys()]